# TODO

* find the varid horizon

In [1]:
import gym, pickle, argparse, json, logging
from itertools import product
from copy import deepcopy
import tensorflow as tf
import ray

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG
from ray.rllib.agents import Trainer
from ray.rllib.evaluation import PolicyEvaluator, SampleBatch, MultiAgentBatch
from ray.rllib.evaluation.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.evaluation.metrics import collect_metrics
from ray.tune.registry import register_env
from ray.rllib.optimizers.rollout import collect_samples
from ray.rllib.optimizers import LocalMultiGPUOptimizer
from ray.rllib.utils.annotations import override

from flow.multiagent_envs import MultiWaveAttenuationPOEnv
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder, get_flow_params
logger = logging.getLogger(__name__)

In [2]:
num_cpus = 3
num_rollouts = 3
horizon = 750
gae_lambda = 0.97
step_size = 5e-4
num_iter = 10
benchmark_name = "multi_merge"
exp_name = "test_ir"

In [3]:
ray.init(num_cpus=num_cpus, logging_level=40, ignore_reinit_error=True)

{'node_ip_address': '169.237.32.118',
 'object_store_address': '/tmp/ray/session_2019-05-25_18-33-34_4703/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-25_18-33-34_4703/sockets/raylet',
 'redis_address': '169.237.32.118:39207',
 'webui_url': None}

In [4]:
config = deepcopy(DEFAULT_CONFIG)
config["num_workers"] = min(num_cpus, num_rollouts)
config["train_batch_size"] = horizon * num_rollouts
config["sample_batch_size"] = horizon / 2
config["use_gae"] = True
config["horizon"] = horizon
config["lambda"] = gae_lambda
config["lr"] = step_size
config["vf_clip_param"] = 1e6
config["num_sgd_iter"] = 10
config['clip_actions'] = False  # FIXME(ev) temporary ray bug
config["model"]["fcnet_hiddens"] = [128, 64, 32]
config["observation_filter"] = "NoFilter"
config["entropy_coeff"] = 0.0

benchmark = __import__(
            "flow.benchmarks.%s" % benchmark_name, fromlist=["flow_params"])
flow_params = benchmark.buffered_obs_flow_params

# save the flow params for replay
flow_json = json.dumps(
    flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
config['env_config']['flow_params'] = flow_json

In [26]:
create_env, env_name = make_create_env(params=flow_params, version=0)
register_env(env_name, create_env)
env = create_env()

default_policy = (PPOPolicyGraph, env.observation_space, env.action_space, {})
policy_graph = {DEFAULT_POLICY_ID: default_policy}
config["multiagent"] = {
        'policy_graphs': policy_graph,
        'policy_mapping_fn': tune.function(lambda agent_id: DEFAULT_POLICY_ID)
    }

In [56]:
class GailTrainer(Trainer):
    _name = "GAIL"
    _default_config = DEFAULT_CONFIG
    _policy_graph = PPOPolicyGraph
    
    @override(Trainer)
    def _init(self, config, env_creator):
        self.local_evaluator = self.make_local_evaluator(
             env_creator, self._policy_graph)        
        self.remote_evaluators = self.make_remote_evaluators(
            env_creator, self._policy_graph, config["num_workers"])
        
        self.standardize_fields = ["advantages"]
        self.sample_batch_size = config["sample_batch_size"]
        self.num_envs_per_worker = config["num_envs_per_worker"]
        self.train_batch_size = config["train_batch_size"]
        self.num_sgd_iter = config["num_sgd_iter"]
        self.sgd_minibatch_size = config["sgd_minibatch_size"]
        
        self.policies = dict(
            self.local_evaluator.foreach_trainable_policy(lambda p, i: (i, p)))
        
    @override(Trainer)    
    def _train(self):
        weights = ray.put(self.local_evaluator.get_weights())
        for e in self.remote_evaluators:
            e.set_weights.remote(weights)       
        
        # collect samples
        samples = collect_samples(
            self.remote_evaluators, self.sample_batch_size,
            self.num_envs_per_worker, self.train_batch_size)
        
        if isinstance(samples, SampleBatch):
            samples = MultiAgentBatch({
                    DEFAULT_POLICY_ID: samples
                }, samples.count)

        for policy_id, policy in self.policies.items():
            if policy_id not in samples.policy_batches:
                continue

            batch = samples.policy_batches[policy_id]
            for field in self.standardize_fields:
                value = batch[field]
                standardized = (value - value.mean()) / max(1e-4, value.std())
                batch[field] = standardized

            # Important: don't shuffle RNN sequence elements
            if not policy._state_inputs:
                batch.shuffle()        
        # training
        for _ in range(self.num_sgd_iter):
            for i in range(0, samples.count, self.sgd_minibatch_size):
                minibatch = samples.slice(i, i+self.sgd_minibatch_size)
                self.local_evaluator.learn_on_batch(minibatch)
        return collect_metrics(remote_evaluators=self.remote_evaluators)


In [57]:
agent = GailTrainer(config, env_name)

2019-05-25 17:55:20,341	INFO policy_evaluator.py:311 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
/opt/conda/envs/flow-latest/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
2019-05-25 17:55:21,868	INFO policy_evaluator.py:728 -- Built policy map: {'default_policy': <ray.rllib.agents.ppo.ppo_policy_graph.PPOPolicyGraph object at 0x7fc097133128>}
2019-05-25 17:55:21,869	INFO policy_evaluator.py:729 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7fc0971339b0>}
2019-05-25 17:55:21,871	INFO policy_evaluator.py:343 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7fc09716d080>}
2019-05-25 17:55:21,964	INFO multi_gpu_optimizer.py:78 -- LocalMul

In [58]:
samples = collect_samples(agent.remote_evaluators, agent.sample_batch_size,
                         agent.num_envs_per_worker, agent.train_batch_size)

AttributeError: 'GailTrainer' object has no attribute 'sample_batch_size'

In [59]:
agent.train()

(pid=2378) 2019-05-25 17:55:58,586	INFO policy_evaluator.py:437 -- Generating sample batch of size 375.0
(pid=2375) Loading configuration... done.
(pid=2375) Success.
(pid=2382) Loading configuration... done.
(pid=2382) Success.
(pid=2378) Loading configuration... done.
(pid=2378) Success.
(pid=2375) Loading configuration... done.
(pid=2382) Loading configuration... done.
(pid=2378) Loading configuration... done.
(pid=2378) 2019-05-25 17:56:00,064	INFO sampler.py:308 -- Raw obs from env: { 0: { 'flow_1.0': np.ndarray((12,), dtype=float32, min=0.0, max=1.0, mean=0.514),
(pid=2378)        'flow_1.1': np.ndarray((12,), dtype=float32, min=0.025, max=0.875, mean=0.2)}}
(pid=2378) 2019-05-25 17:56:00,064	INFO sampler.py:309 -- Info return from env: {0: {'flow_1.0': {}, 'flow_1.1': {}}}
(pid=2378) 2019-05-25 17:56:00,065	INFO sampler.py:407 -- Preprocessed obs: np.ndarray((12,), dtype=float32, min=0.025, max=0.875, mean=0.2)
(pid=2378) 2019-05-25 17:56:00,065	INFO sampler.py:411 -- Filtered o

(pid=2375) Loading configuration... done.
(pid=2382) Loading configuration... done.
(pid=2382) Success.
(pid=2382) Loading configuration... done.
(pid=2378) 2019-05-25 17:56:04,661	INFO policy_evaluator.py:474 -- Completed sample batch:
(pid=2378) 
(pid=2378) { 'data': { 'action_prob': np.ndarray((843,), dtype=float32, min=0.003, max=0.4, mean=0.279),
(pid=2378)             'actions': np.ndarray((843, 1), dtype=float32, min=-3.093, max=3.14, mean=0.013),
(pid=2378)             'advantages': np.ndarray((843,), dtype=float32, min=-22.849, max=-0.302, mean=-17.036),
(pid=2378)             'agent_index': np.ndarray((843,), dtype=int64, min=0.0, max=4.0, mean=2.396),
(pid=2378)             'behaviour_logits': np.ndarray((843, 2), dtype=float32, min=-0.004, max=0.002, mean=-0.001),
(pid=2378)             'dones': np.ndarray((843,), dtype=bool, min=0.0, max=1.0, mean=0.006),
(pid=2378)             'eps_id': np.ndarray((843,), dtype=int64, min=1965020631.0, max=1965020631.0, mean=1965020631.0)

{'config': {'batch_mode': 'complete_episodes',
  'callbacks': {'on_episode_end': None,
   'on_episode_start': None,
   'on_episode_step': None,
   'on_postprocess_traj': None,
   'on_sample_end': None,
   'on_train_result': None},
  'clip_actions': False,
  'clip_param': 0.3,
  'clip_rewards': None,
  'collect_metrics_timeout': 180,
  'compress_observations': False,
  'custom_resources_per_worker': {},
  'entropy_coeff': 0.0,
  'env': 'MultiWaveAttenuationMergePOEnvBufferedObs-v0',
  'env_config': {'flow_params': '{\n    "env": {\n        "additional_params": {\n            "FLOW_RATE": 2000,\n            "FLOW_RATE_MERGE": 100,\n            "RL_PENETRATION": 0.1,\n            "buf_length": 1,\n            "eta1": 1.0,\n            "eta2": 0.2,\n            "eta3": 0.1,\n            "max_accel": 3,\n            "max_decel": 3,\n            "reward_scale": 1.0,\n            "t_min": 1.0,\n            "target_velocity": 25\n        },\n        "evaluate": false,\n        "horizon": 750,\

In [7]:
tune.run_experiments({
    exp_name: {
        "run": GailTrainer,
        "env": env_name,
        "checkpoint_freq": 5,
        "max_failures": 999,
        "num_samples": 1,
        "stop": {
            "training_iteration": num_iter
        },
        "config": config
    }   
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.7/33.4 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.7/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING



2019-05-25 06:36:55,542	ERROR trial_runner.py:494 -- Error processing event.
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 443, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/ray_trial_executor.py", line 315, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/worker.py", line 2193, in get
    raise value
ray.exceptions.RayTaskError: ray_GailTrainer:train() (pid=8953, host=kronos)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/rllib/agents/trainer.py", line 293, in __init__
    Trainable.__init__(self, config, logger_creator)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trainable.py", line 88, in __init__
    self._setup(copy.deepcopy(self.config))
  File "/opt/conda/envs/flow-latest/lib/py

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.9/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING, 1 failures: /headless/ray_results/test_ir/GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0_2019-05-25_06-36-45y51s7ud9/error_2019-05-25_06-36-55.txt



2019-05-25 06:37:05,623	ERROR trial_runner.py:494 -- Error processing event.
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 443, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/ray_trial_executor.py", line 315, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/worker.py", line 2193, in get
    raise value
ray.exceptions.RayTaskError: ray_GailTrainer:train() (pid=8952, host=kronos)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/rllib/agents/trainer.py", line 293, in __init__
    Trainable.__init__(self, config, logger_creator)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trainable.py", line 88, in __init__
    self._setup(copy.deepcopy(self.config))
  File "/opt/conda/envs/flow-latest/lib/py

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.9/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING, 2 failures: /headless/ray_results/test_ir/GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0_2019-05-25_06-36-45y51s7ud9/error_2019-05-25_06-37-05.txt



2019-05-25 06:37:15,741	ERROR trial_runner.py:494 -- Error processing event.
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trial_runner.py", line 443, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/ray_trial_executor.py", line 315, in fetch_result
    result = ray.get(trial_future[0])
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/worker.py", line 2193, in get
    raise value
ray.exceptions.RayTaskError: ray_GailTrainer:train() (pid=8949, host=kronos)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/rllib/agents/trainer.py", line 293, in __init__
    Trainable.__init__(self, config, logger_creator)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/ray/tune/trainable.py", line 88, in __init__
    self._setup(copy.deepcopy(self.config))
  File "/opt/conda/envs/flow-latest/lib/py

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 1.9/33.4 GB
Result logdir: /headless/ray_results/test_ir
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0:	RUNNING, 3 failures: /headless/ray_results/test_ir/GailTrainer_MultiWaveAttenuationMergePOEnvBufferedObs-v0_0_2019-05-25_06-36-45y51s7ud9/error_2019-05-25_06-37-15.txt



Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 177, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/client.py", line 2408, in _execute
    return command(*args)
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 624, in read_response
    response = self._parser.read_response()
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 284, in read_response
    response = self._buffer.readline()
  File "/opt/conda/envs/flow-latest/lib/python3.5/site-packages/redis/connection.py", line 216, in readline
    self._read_from_socket()
  File "/opt/conda/envs/flow-latest/l

KeyboardInterrupt: 